# 6. Решение уравнения теплопроводности методом сеток

In [1]:
from math import cos
from pandas import DataFrame as df

In [2]:
def a(x, t):
    return cos(x)

def b(x, t):
    return x

def c(x, t):
    return 0

alpha1 = 0
alpha2 = -1
beta1 = 1
beta2 = 1

In [3]:
def u(x, t):
    return x**3 + t**3

def dudx(x, t):
    return 3*(x**2)

def dudt(x, t):
    return 3*(t**2)

def d2udx2(x, t):
    return 6 * x

def phi(x):
    return u(x, 0)

def alpha(t):
    return alpha1 * u(0, t) - alpha2 * dudx(0, t)

def beta(t):
    return beta1 * u(1, t) + beta2 * dudx(1, t)

def f(x, t):
    return dudt(x, t) - a(x, t) * d2udx2(x, t) - b(x, t) * dudx(x, t) - c(x, t) * u(x, t)

## Явная разностная схема

In [4]:
# подбирает m из условия устойчивости 5m >= n^2
def calc_m(n):
    m = 5
    while 5 * m < n * n:
        m *= 2
    return m

In [5]:
def solve_explicit(n, m):
    h = 1/n
    tau = 0.1/m
    x = [i * h for i in range(n+1)]
    u = [[0 for i in range(m+1)] for j in range(n+1)]

    for i in range(n+1):
        u[i][0] = phi(x[i])
    
    def L(i, k, h):
        return a(x[i], tau * k) * (u[i + 1][k] - 2 * u[i][k] + u[i - 1][k]) / (h ** 2) + b(x[i], tau * k) * \
               (u[i + 1][k] - u[i - 1][k]) / (2 * h) + c(x[i], tau * k) * u[i][k]
    
    for k in range(1, m+1):
        for i in range(1, n):
            u[i][k] = u[i][k-1] + tau * (L(i, k - 1, h) + f(x[i], tau * (k - 1)))
        u[0][k] = (alpha(tau * k) + alpha2 * (4 * u[1][k] - u[2][k]) / (2 * h)) / (alpha1 + 3 * alpha2 / (2 * h))
        u[n][k] = (beta(tau * k) - beta2 * (-4 * u[n - 1][k] + u[n - 2][k]) / (2 * h)) / (beta1 + 3 * beta2 / (2 * h))
    
    return u                        

In [6]:
N = 5
M = calc_m(5)
N, M

(5, 5)

In [7]:
solution = solve_explicit(N, M)
table = df(data=[[solution[i * N // 5][j * M // 5] for j in range(6)] for i in range(6)], index=[i / 5 for i in range(6)], columns=[i * 0.1 / 5 for i in range(6)])
table

,0.00,0.02,0.04,0.06,0.08,0.10
0.0,0.000,-0.010560,-0.017186,-0.020626,-0.023848,-0.025852
0.2,0.008,0.008160,0.003278,0.000247,-0.002247,-0.003830
0.4,0.064,0.064320,0.064670,0.062866,0.062557,0.062236
0.6,0.216,0.216480,0.216994,0.219175,0.220496,0.222168
0.8,0.512,0.512640,0.516899,0.520311,0.523455,0.526283
1.0,1.000,1.010024,1.014891,1.018281,1.021626,1.024519


In [8]:
N = 10
M = calc_m(10)
N, M

(10, 20)

In [9]:
solution = solve_explicit(N, M)
table = df(data=[[solution[i * N // 5][j * M // 5] for j in range(6)] for i in range(6)], index=[i / 5 for i in range(6)], columns=[i * 0.1 / 5 for i in range(6)])
table

,0.00,0.02,0.04,0.06,0.08,0.10
0.0,0.000,-0.003122,-0.004333,-0.005120,-0.005575,-0.005694
0.2,0.008,0.007508,0.006659,0.006072,0.005770,0.005777
0.4,0.064,0.064086,0.063934,0.063842,0.063942,0.064283
0.6,0.216,0.216126,0.216455,0.216916,0.217500,0.218247
0.8,0.512,0.512486,0.513387,0.514297,0.515244,0.516289
1.0,1.000,1.002617,1.003765,1.004763,1.005761,1.006841


In [10]:
N = 20
M = calc_m(20)
N, M

(20, 80)

In [11]:
solution = solve_explicit(N, M)
table = df(data=[[solution[i * N // 5][j * M // 5] for j in range(6)] for i in range(6)], index=[i / 5 for i in range(6)], columns=[i * 0.1 / 5 for i in range(6)])
table

,0.00,0.02,0.04,0.06,0.08,0.10
0.0,0.000,-0.000780,-0.001040,-0.001121,-0.001012,-0.000674
0.2,0.008,0.007861,0.007703,0.007675,0.007824,0.008193
0.4,0.064,0.064016,0.064023,0.064117,0.064366,0.064818
0.6,0.216,0.216042,0.216162,0.216388,0.216755,0.217308
0.8,0.512,0.512135,0.512395,0.512734,0.513192,0.513819
1.0,1.000,1.000651,1.000985,1.001351,1.001824,1.002460


## Схема с весами

In [12]:
def solve_weights(n, m):
    h = 1/n
    tau = 0.1/m
    x = [i * h for i in range(n+1)]
    u = [[0 for i in range(m+1)] for j in range(n+1)]
    t = [i * tau for i in range(m+1)]
    sigma = 1/2
    
    for i in range(n + 1):
        u[i][0] = phi(x[i])
        
    def L(i, k, h):
        return a(x[i], tau * k) * (u[i + 1][k] - 2 * u[i][k] + u[i - 1][k]) / (h ** 2) + b(x[i], tau * k) * \
               (u[i + 1][k] - u[i - 1][k]) / (2 * h) + c(x[i], tau * k) * u[i][k]

    for k in range(1, m+1):
        A = [0]
        B = [-alpha1 - alpha2 / h]
        C = [-alpha2 / h]
        G = [alpha(t[k])]
        for i in range(1, n):
            A.append(sigma * a(x[i], t[k]) / (h ** 2) - sigma * b(x[i], t[k]) / (2 * h))
            B.append(2 * sigma * a(x[i], t[k]) / (h ** 2) - sigma * c(x[i], t[k]) + 1 / tau)
            C.append(sigma * a(x[i], t[k]) / (h ** 2) + sigma * b(x[i], t[k]) / (2 * h))
            G.append(-u[i][k - 1] / tau - (1 - sigma) * L(i, k - 1, h) - f(x[i], t[k] - tau / 2))
        A.append(-beta2 / h)
        B.append(-beta1 - beta2 / h)
        C.append(0)
        G.append(beta(tau * k))
        
        S = [0] * len(A)
        T = [0] * len(A)
        
        S[0] = C[0] / B[0]
        for i in range(1, len(A)):
            S[i] = C[i] / (B[i] - A[i] * S[i-1])
    
        T[0] = -G[0] / B[0]
        for i in range(1, len(A)):
            T[i] = (A[i] * T[i-1] - G[i]) / (B[i] - A[i] * S[i-1])
        
        u[len(A) - 1][k] = T[len(A) - 1]
        for i in range(len(A) - 2, -1, -1):
            u[i][k] = S[i] * u[i+1][k] + T[i]
        
    return u

In [13]:
N = 5
M = calc_m(5)
N, M

(5, 5)

In [14]:
solution = solve_explicit(N, M)
table = df(data=[[solution[i * N // 5][j * M // 5] for j in range(6)] for i in range(6)], index=[i / 5 for i in range(6)], columns=[i * 0.1 / 5 for i in range(6)])
table

,0.00,0.02,0.04,0.06,0.08,0.10
0.0,0.000,-0.010560,-0.017186,-0.020626,-0.023848,-0.025852
0.2,0.008,0.008160,0.003278,0.000247,-0.002247,-0.003830
0.4,0.064,0.064320,0.064670,0.062866,0.062557,0.062236
0.6,0.216,0.216480,0.216994,0.219175,0.220496,0.222168
0.8,0.512,0.512640,0.516899,0.520311,0.523455,0.526283
1.0,1.000,1.010024,1.014891,1.018281,1.021626,1.024519


In [15]:
N = 10
M = calc_m(10)
N, M

(10, 20)

In [16]:
solution = solve_explicit(N, M)
table = df(data=[[solution[i * N // 5][j * M // 5] for j in range(6)] for i in range(6)], index=[i / 5 for i in range(6)], columns=[i * 0.1 / 5 for i in range(6)])
table

,0.00,0.02,0.04,0.06,0.08,0.10
0.0,0.000,-0.003122,-0.004333,-0.005120,-0.005575,-0.005694
0.2,0.008,0.007508,0.006659,0.006072,0.005770,0.005777
0.4,0.064,0.064086,0.063934,0.063842,0.063942,0.064283
0.6,0.216,0.216126,0.216455,0.216916,0.217500,0.218247
0.8,0.512,0.512486,0.513387,0.514297,0.515244,0.516289
1.0,1.000,1.002617,1.003765,1.004763,1.005761,1.006841


In [17]:
N = 20
M = calc_m(20)
N, M

(20, 80)

In [18]:
solution = solve_explicit(N, M)
table = df(data=[[solution[i * N // 5][j * M // 5] for j in range(6)] for i in range(6)], index=[i / 5 for i in range(6)], columns=[i * 0.1 / 5 for i in range(6)])
table

,0.00,0.02,0.04,0.06,0.08,0.10
0.0,0.000,-0.000780,-0.001040,-0.001121,-0.001012,-0.000674
0.2,0.008,0.007861,0.007703,0.007675,0.007824,0.008193
0.4,0.064,0.064016,0.064023,0.064117,0.064366,0.064818
0.6,0.216,0.216042,0.216162,0.216388,0.216755,0.217308
0.8,0.512,0.512135,0.512395,0.512734,0.513192,0.513819
1.0,1.000,1.000651,1.000985,1.001351,1.001824,1.002460
